<a href="https://colab.research.google.com/github/AlexEMG/DeepLabCut/blob/master/examples/COLAB_DLC_ModelZoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DeepLabCut Model Zoo:**

http://www.mousemotorlab.org/dlc-modelzoo

You can use this notebook to analyze videos with pretrained networks from our model zoo - NO local installation of DeepLabCut is needed! 

- **What you need:** a video of your favorite dog, cat, etc: check the list of currently available models here: http://www.mousemotorlab.org/dlc-modelzoo 

- **What to do:** hit run on each cell below and follow the instructions!

- **If you performance is less that you would like:** firstly check the labeled_video parameters (i.e. "pcutoff" in the config.yaml file that will set the video plotting) - see the end of this notebook. 

  * *Otherwise, please consider labeling data to help make each network even better! See instructions here: http://www.mousemotorlab.org/dlc-modelzoo - this is our great experiment on community-sourced open-science!*


Let's get going! 

## **Let's install DeepLabCut into COLAB:**

*Also, be sure you are connected to a GPU: go to menu, click Runtime > Change Runtime Type > select "GPU"*

In [0]:
#click the play icon (this will take a few minutes to install all the dependences!)
!pip install deeplabcut
%reload_ext numpy
%reload_ext matplotlib
%reload_ext mpl_toolkits

In [0]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

## Now let's set the backend & import the DeepLabCut package:

In [0]:
#GUIs don't work on the cloud, so we supress them:
import os
os.environ["DLClight"]="True"

# stifle tensorflow warnings, like we get it already.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import deeplabcut

## Next, run the cell below to upload your video file from your computer:

In [0]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{video_path}" with length {length} bytes'.format(
      video_path=fn, length=len(uploaded[fn])))
  video_path = fn

## Select your model from the dropdown menu, then below (optionally) input the name you want for the project:


In [0]:
import ipywidgets as widgets
from IPython.display import display
model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = widgets.Dropdown(
    options=model_options,
    value=model_options[0],
    description="Choose a DLC ModelZoo model!",
    disabled=False
)
display(model_selection)

In [0]:
ProjectFolderName = 'myDLC_modelZoo'
YourName = 'teamDLC'
model2use = model_selection.value # see http://www.mousemotorlab.org/dlc-modelzoo for the list! (curently: full_dog, full_cat, full_human, primate_face)
videotype = os.path.splitext(video_path)[-1].lstrip('.') #or MOV, or avi, whatever you uploaded!

## Attention on this step !! 
- Please note that for optimal performance your videos should contain frames that are around ~300-600 pixels (on one edge). If you have a larger video (like from an iPhone, first downsize by running this please! :)

- Thus, if you're using an iPhone, or such, you'll need to downsample the video first by running the code below**

(no need to edit it unless you want to change the size)

In [0]:
deeplabcut.DownSampleVideo(video_path, width=300)

import os
from pathlib import Path
video_path=os.path.join(str(Path(video_path).stem)+'downsampled.'+videotype)
print(video_path)

## Lastly, run the cell below to create a pretrained project, analyze your video with your selected pretrained network, plot trajectories, and create a labeled video!:


In [0]:
path_config_file = deeplabcut.create_pretrained_project(ProjectFolderName, YourName, video_path, videotype=videotype, 
                                      model=model2use, analyzevideo=True, createlabeledvideo=True, copy_videos=True) #must leave copy_videos=True

Now, you can move this project from Colab (i.e. download it to your GoogleDrive), and use it like a normal standard project! 

You can analyze more videos, extract outliers, refine then, and/or then add new key points + label new frames, and retrain if desired. We hope this gives you a good launching point for your work!

###Happy DeepLabCutting! Welcome to the Zoo :)



## More advanced options: 

- If you would now like to customize the video/plots - i.e., color, dot size, threshold for the point to be plotted (pcutoff), please simply edit the "config.yaml" file by updating the values below:

In [0]:
# Updating the plotting within the config.yaml file (without opening it ;):

#dotsize: size of the dots!
#colormap: any matplotlib colormap!
#pcutoff: the higher the more conservative the plotting!

config_path = path_config_file[0]
edits = {'dotsize': 7,
          'colormap': 'spring',
          'pcutoff': 0.5}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

In [0]:
# re-create the labeled video (first you will need to delete in the folder to the LEFT!):
from datetime import datetime
#The name of the project you created:
project_folder_name = '-'.join([ProjectFolderName, YourName, datetime.now().strftime('%Y-%m-%d')])

full_video_path = videofile_path = ['/content/'+project_folder_name+'/videos/'+video_path]

#filter predictions (should already be done above ;):
deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

#re-create the video with your edits!
deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, filtered=True)